# Import

In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import statsmodels.api as sm
from config import constants
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from helper.helper import Helper

In [ ]:
%matplotlib inline

In [ ]:
# disable division by zero warning
np.seterr(divide='ignore', invalid='ignore')

# disabling max open windows warning in matplotlib
matplotlib.rcParams.update({'figure.max_open_warning': 0})

# deactivate SettingWithCopyWarning:
pd.options.mode.chained_assignment = None

# Read raw data file

In [ ]:
df = pd.read_excel(constants.SOURCE_FILE, sheet_name=constants.XLSX_SHEET)

# Preparation

In [ ]:
# drop rows with null ids
df.dropna(subset=['store_nbr', 'station_nbr', 'item_nbr'], inplace=True)
# delete lines where units greather than 0
df = df[df['units'] > 0]
# create new df with only relevant columns
relevant_columns = ['date', 'item_nbr', 'station_nbr', 'store_nbr', 'units', 'tmin', 'tmax', 'tavg', 'wetbulb', 'dewpoint', 'snowfall', 'preciptotal', 'avgspeed']
df_new = df[relevant_columns]
# calculate humidity
df_new['humidity'] = df_new.apply(func=Helper.calculate_relative_humidity_jupyter, axis=1)

# Fetch Store and Item IDs

In [ ]:
stores = df_new['store_nbr'].unique()
items = df_new['item_nbr'].unique()
dates = df_new['date'].unique()

# Detect significant Attributes with Multiple Regression

In [ ]:
#fetch all rows from item per store
significant_columns = []

for store_id in stores:
    for item_id in items:
        df_reg = df_new[(df_new['store_nbr'] == store_id) & (df_new['item_nbr'] == item_id)]
        if df_reg.shape[0] > 0:
            df_reg.fillna(0, inplace=True)
            feature_columns = ['snowfall', 'preciptotal', 'tavg', 'tmin', 'tmax', 'humidity', 'avgspeed']
            x = df_reg[feature_columns]
            y = df_reg[['units']]
            # x = sm.add_constant(x)
            model = sm.OLS(y, x).fit()
            p_values = model.pvalues
            for column in feature_columns:
                p_value = float(p_values[column])
                if (not np.isnan(p_value)) and (p_value < constants.SIGNIFICANT_LIMIT):
                    summary = (store_id, item_id, column, p_value, model)
                    significant_columns.append(summary)

In [ ]:
significant_columns

# Checking R2 Adjusted

In [ ]:
for summary_tuple in significant_columns:
    column_model = summary_tuple[4]
    column_name = summary_tuple[2]
    print(column_model.rsquared_adj)

# Plotting Regression Plots

In [ ]:
for summary_tuple in significant_columns:
    column_model = summary_tuple[4]
    column_name = summary_tuple[2]
    fig = plt.figure(figsize=(15,8))
    fig = sm.graphics.plot_regress_exog(column_model, column_name, fig=fig)

# Correlation instead of regression

In [ ]:
high_correlation = []

for store_id in stores:
    for item_id in items:
        df_reg = df_new[(df_new['store_nbr'] == store_id) & (df_new['item_nbr'] == item_id)]
        if df_reg.shape[0] > 0:
            df_reg.fillna(0, inplace=True)
            feature_columns = ['snowfall', 'preciptotal', 'tavg', 'tmin', 'tmax', 'humidity', 'avgspeed', 'units']
            corr_df = df_reg[feature_columns]
            corr = corr_df.corr(method='spearman')
            for a in feature_columns:
                for b in feature_columns:
                    if (a == 'units') or (b == 'units'):
                        correlation_value = corr[a][b]
                        if (not np.isnan(correlation_value)) and (abs(correlation_value) > constants.HIGH_CORRELATION) and (abs(correlation_value) != 1):
                            summary_tuple = (store_id, item_id, a, b, correlation_value, corr)
                            high_correlation.append(summary_tuple)

# Plotting correlation with Heatmap

In [ ]:
for correlation_tuple in high_correlation:
    corr = correlation_tuple[5]
    fig, ax = plt.subplots(figsize=(5, 5))
    mappable = ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=60);
    plt.yticks(range(len(corr.columns)), corr.columns);
    plt.colorbar(mappable)

# Plotting correlation with Scatter

In [ ]:
for correlation_tuple in high_correlation:
    store_id = correlation_tuple[0]
    item_id = correlation_tuple[1]
    column_a = correlation_tuple[2]
    column_b = correlation_tuple[3]
    corr_value = correlation_tuple[4]
    print(store_id, item_id, column_a, column_b, corr_value)
    df_corr = df_new[(df_new['store_nbr'] == store_id) & (df_new['item_nbr'] == item_id)]
    if column_a == 'units':
        x = df_corr[column_b]
        y = df_corr[column_a]
    else:
        x = df_corr[column_a]
        y = df_corr[column_b]
    plt.scatter(x, y)
    plt.show()

# Plotting trends for significant columns

In [ ]:
# TODO
# fetching units per item/store
            if trend_df.shape[0] > 0:
                trend_df.fillna(0, inplace=True)
                trend_df['date'] = trend_df['date'] + 1
                trend_df.rename(index=str, columns={"date": "day"}, inplace=True)
                trend_df.set_index('day', inplace=True)
                trend_df.plot(figsize=(8, 5), linewidth=4, fontsize=12)
                plt.xlabel('Day', fontsize=12)
                y_label = 'Item ' + str(key) + ' in Store ' + str(store_id[0])
                plt.ylabel(y_label, fontsize=15)